# Machine learning on mobile
<img src="files/catvsdog.png" />
The goal is to get a trained image classifier working on your phone. The image classifier should be able to detect if an image is a cat or a dog and what percentage cat or dog the picture is.

Steps needed to complete this project.

1. Gather data for training a machine learning model and clean up the data.
2. Traing an image classifier using Keras.
3. Create a restful API that we can send a request to that uses our trained model.
4. Build a mobile application that can take a picture and classify how much cat or dog a picture is

This tutorial is inspired by this Keras example.
https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

Here is an image of the entire machine learning data process.
<img src="files/datasteps.png" />


# Gather data for training a machine learning model
<img src="files/puppy.jpg" />
In this tutorial, we will present a few simple yet effective methods that you can use to build a powerful image classifier, using only very few training examples --just a few hundred or thousand pictures from each class you want to be able to recognize.

We will go over the following options:

* how to structure training and test data
* training a small network from scratch (as a baseline) on image data
* how to improve model predictions

To acquire a few hundreds or thousands of training images belonging to the classes you are interested in, one possibility would be to use the Flickr API to download pictures matching a given tag, under a friendly license.

In our examples we will use two sets of pictures, which we got from Kaggle: 1000 cats and 1000 dogs (although the original dataset had 12,500 cats and 12,500 dogs, we just took the first 1000 images for each class). We also use 400 additional samples from each class as validation data, to evaluate our models.

That is very few examples to learn from, for a classification problem that is far from simple. So this is a challenging machine learning problem, but it is also a realistic one: in a lot of real-world use cases, even small-scale data collection can be extremely expensive or sometimes near-impossible (e.g. in medical imaging). Being able to make the most out of very little data is a key skill of a competent data scientist.

1. The data can be downloaded from kaggle. Because we are awesome we also stored the data on our floydhob server
https://www.floydhub.com/viewer/data/xLHRt9d9UnipGEEr4FkWdF/

https://www.kaggle.com/c/dogs-vs-cats
    
 
Step 1.
Download the data from kaggle and sort the data into two different folders
Train containing 1000 cat images and 1000 dog images and Validation containing 400 images

Here is how you should structure your folders.
<img src="files/folderstructure.png" />


In [ ]:
# preprocessing
import os
import errno
import itertools
import scipy.misc
import numpy as np

curr_dir = os.getcwd()
proj_dir = os.path.normpath(os.path.join(curr_dir))
image_dir = 'data/train'
input_filepath = os.path.normpath(os.path.join(proj_dir, image_dir))

validation_dir = 'data/processed/validation'
train_dir = 'data/processed/train'

class_dict = {'dog': 0, 'cat': 1}

def put_classes_into_separate_folders(parent_dir, images):
    make_sure_path_exists(parent_dir)

def make_sure_path_exists(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise


train_shuffled_images = []
test_shuffled_images = []
def create_train_validation_shuffle_images():
    test_percentage = 0.2
    for key, _ in class_dict.items():
        class_images = [x for x in os.listdir(input_filepath) if x.startswith(key + ".")]
        k = int(len(class_images) * test_percentage)
        test_shuffled_images = class_images[0:k]
        train_shuffled_images = class_images[k:]
    return train_shuffled_images, test_shuffled_images

train_images = []
validation_images = []
def resize_of_images():
    train_images = [scipy.misc.imresize(scipy.misc.imread(input_filepath + '/' + image), (100,50)) for image in train_shuffled_images]
    validation_images = [scipy.misc.imresize(scipy.misc.imread(input_filepath +
        '/' + image), (100,50)) for image in test_shuffled_images]
    return train_images, validation_images

def put_class_images_in_folders(save_dir, image_files, class_feature):
    counter = 0
    class_counter = 0
    first = True

    make_sure_path_exists(save_dir)

    for image in image_files:
        counter += 1

        if (counter % int(len(image_files) / len(class_feature)) == 0) and (first == False) != (counter == int(len(image_files))):
            class_counter += 1
        first = False

        class_dir = os.path.join(save_dir,class_feature[class_counter])
        make_sure_path_exists(class_dir)

        save_image = os.path.join(class_dir,'{}_{}.jpeg'.format(class_feature[class_counter],"".join((map(str,np.random.randint(0,9,8))))))

        scipy.misc.imsave(save_image, image)


if __name__ == '__main__':
    train_shuffled_images, test_shuffled_images = create_train_validation_shuffle_images()
    train_images, test_images = resize_of_images()
    keys = []
    for k, _ in class_dict.items():
        keys.append(k)
    print(input_filepath)
    put_class_images_in_folders(input_filepath + '/train/', train_images,
            keys)
    put_class_images_in_folders(input_filepath + '/validation/', test_images,
            keys)

# Convolutional neural networks
<img src="files/visualcortex.png" />
<img src="files/convolution.jpg" />

Convolutional neural networks are neural network that are specialised in image classification. Their functioning is inspired by the mammalian visual cortex.

In [10]:
'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

# flowdhub folder
flowdhub = 'https://www.floydhub.com/viewer/data/xLHRt9d9UnipGEEr4FkWdF/'
flowhubtrain = 'https://www.floydhub.com/viewer/data/xLHRt9d9UnipGEEr4FkWdF/train'
flowdhubvalidation = 'https://www.floydhub.com/viewer/data/xLHRt9d9UnipGEEr4FkWdF/validation'

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 2
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')
saved_model = model.to_json()
with open('first_try.json','w') as f:
    f.write(saved_model)

Found 2002 images belonging to 2 classes.
Found 801 images belonging to 2 classes.
Epoch 1/2
125/125 [==============================] - 71s - loss: 0.6986 - acc: 0.5530 - val_loss: 0.6580 - val_acc: 0.6025

In [11]:
# Saving the model weights
model.save_weights('first_try.h5')
saved_model = model.to_json()
with open('first_try.json','w') as f:
    f.write(saved_model)

In [12]:
# Loading the saved model
from keras.models import model_from_json
# Load trained model
# load json and create model
json_file = open('first_try.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
global loaded_model
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("first_try.h5")
print("Loaded model from disk")

Loaded model from disk


In [36]:
# Reshaping our trained image
import scipy.misc

testimage = scipy.misc.imresize(scipy.misc.imread('data/validation/cats/cat.1211.jpg'), (150, 150))
testimage = testimage.reshape((1,) + testimage.shape)

# checking the shape of our new image
testimage.shape

(1, 150, 150, 3)

# # Lets test our prediction on a test image
<img src="data/validation/cats/cat.1211.jpg" />

In [37]:
prediction = loaded_model.predict(testimage)
print(prediction)

if (prediction[0] == 1):
        print("Cat")
        one = True
else:
    print("Dog")


[[ 1.]]
Cat


# Our classification works but it isn't that good.
What can we do to improve our predictions? Hint. You could probably change settings for how you train your model.

# Create a restful API to serve the trained Keras Model and store it on Heroku

# Building a AI web app
Machine learning is it's own paradigm that is complicated to learn. However even if you learn everything about machine learning you need to figure out a way to get your machine learning model up and working online.

There are a few different ways of doing that.

The easiests is using a small python library, while other frameworks can be helpful for web applications



# Flask mini framework
<img src="files/flask.png" />
Flask is a micro framework for creating web server. With it you can create a web application that serves up your trained machine learning models in an API.


# Trained keras model
To be able to serve a model we need to train a model 
that can be returned with an API call. We have already trained our model. So now our job is to load our trained model. 

Then we will build an API that returns neural networks best guess of what the image is depicting, a cat or a dog.


## Building an app in Flask
The example code below is NOT a functioning Flask Application. It is however a working rest API for returning tasks which might help you get started building your API. 

Here you can learn more about flask
http://flask.pocoo.org/

It is your job to find a way to make it work.
You are of course allowed to try other methods of making it work. Tensorflow serving is one method.
https://www.tensorflow.org/serving/

In [ ]:
# simple code for a python API using flask
#!flask/bin/python
from flask import Flask, jsonify

app = Flask(__name__)

tasks = [
    {
        'id': 1,
        'title': u'Buy groceries',
        'description': u'Milk, Cheese, Pizza, Fruit, Tylenol',
        'done': False
    },
    {
        'id': 2,
        'title': u'Learn Python',
        'description': u'Need to find a good Python tutorial on the web',
        'done': False
    }
]


from flask import abort

# getting an individual task
@app.route('/todo/api/v1.0/tasks/<int:task_id>', methods=['GET'])
def get_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    return jsonify({'task': task[0]})

# getting all the tasks
@app.route('/todo/api/v1.0/tasks', methods=['GET'])
def get_tasks():
    return jsonify({'tasks': tasks})

from flask import make_response

# error handling
@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': 'Not found'}), 404)

from flask import request

@app.route('/todo/api/v1.0/tasks', methods=['POST'])
def create_task():
    if not request.json or not 'title' in request.json:
        abort(400)
    task = {
        'id': tasks[-1]['id'] +1,
        'title': request.json['title'],
        'description': request.json.get('description', ""),
        'done': False
    }
    tasks.append(task)
    return jsonify({'task': task}), 201

@app.route('/todo/api/v1.0/tasks/<int:task_id>', methods=['PUT'])
def update_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    if not request.json:
        abort(400)
    if 'title' in request.json and type(request.json['title']) != unicode:
        abort(400)
    if 'description' in request.json and type(request.json['done']) is not bool:
        abort(400)
    if 'done' in request.json and type(request.json['done']) is not bool:
        abort(400)
    task[0]['title'] = request.json.get('title', task[0]['title'])
    task[0]['description'] = request.json.get('description', task[0]['description'])
    task[0]['done'] = request.json.get('done', task[0]['done'])
    return jsonify({'task': task[0]})

@app.route('/todo/api/v1.0/tasks/<int:task_id>', methods=['DELETE'])
def delete_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    tasks.remove(task[0])
    return jsonify({'result': True})

if __name__ == '__main__':
    app.run(debug=True)


# Build a mobile application that can take a picture and classify how much cat or dog a picture is
<img src="files/catvsdog.png" />

There is a lot of moving parts that needs to fit together in order to get a machine learning model working on a mobile phone. Each domain is hard and getting it all together is tricky. Still. We should try to make everything work.

In order to do that we will use React Native. A framework made by Google that let us use Javascript to build native applications for iOS and Android.

https://facebook.github.io/react-native/

## Getting started with React Native
<img src="files/reactnative.png" />
Start by installing React Native from the terminal:
npm install -g create-react-native-app

Secondly, download Expo, an integrative environment on your phone. 
https://expo.io/

By using Expo we can get an app running on any phone that got the expo APP installed. 

You can even use it live on the web.
https://snack.expo.io

<img src="files/reactnativeexpo.png" />


## Using Snack Expo
Snack expo is an amazing tool that let you load a react native project inside your browser that can run on your phone.

To get started with Expo you need to download Expo on your phone and then scan the QR-code from the project.
<img src="files/getstartedwithexpo.png" />
Once you have scanned the project the project will update on your phone in real time and code changes will be automatically posted to your phone.

The sample code can post a picture but it has no connection to the rest API that needs to be constructed based on your trained models.

Look into the sample code and see if you can figure out a way to change the code so that you can return values from your machine learning model. 

In [ ]:
## example Javascript code for the React app. 
## Paste this code into https://snack.expo.io to get started
import React from 'react';
import {
  ActivityIndicator,
  Button,
  Clipboard,
  Image,
  Share,
  StatusBar,
  StyleSheet,
  Text,
  View,
  ScrollView
} from 'react-native';
import { ImagePicker } from 'expo';

export default class App extends React.Component {
  state = {
    image: null,
    uploading: false,
    imageclass: false
  };

  getFbData(){
  return fetch('https://facebook.github.io/react-native/movies.json')
    .then((response) => response.json())
    .then((movies) => {
      this.setState({movies})
      //alert(movies.movies[0].title);
      alert("50% dog")
    })
    .catch((error) => {
      console.error(error);
    });
}

componentDidMount(){
  //this.getFbData();
}





  render() {

    return (
      <ScrollView style={{ flex: 1, alignItems: 'center', justifyContent: 'center' }}>

        <Text
          style={{
            fontSize: 20,
            marginBottom: 20,
            textAlign: 'center',
            marginHorizontal: 15,
          }}>
          Cat vs Dog
        </Text>
    
        <Image
          source={{ uri: 'http://d23dyxeqlo5psv.cloudfront.net/cat.gif' }}
          style={{ height: 140, width: 200 }}
        />
    
    
       
        <Image
          source={{ uri: 'https://media.giphy.com/media/l0HlIOM2MTxoB6tI4/giphy.gif' }}
          style={{ height: 140, width: 200 }}
        />
      
      

        <Button
          onPress={this._pickImage}
          title="Pick an image from camera roll"
        />

        <Button onPress={this._takePhoto} title="Take a photo" />

        {this._maybeRenderImage()}
        {this._maybeRenderUploadingOverlay()}

        <StatusBar barStyle="default" />
      </ScrollView>
    );
  }

  _maybeRenderResultOverlay = () => {
    if (this.state.imageclass) {
      return (
        <View
          style={[
            StyleSheet.absoluteFill,
            {
              backgroundColor: 'rgba(0,0,0,0.4)',
              alignItems: 'center',
              justifyContent: 'center',
            },
          ]}>
          <Text
          style={{
            fontSize: 20,
            marginBottom: 20,
            textAlign: 'center',
            marginHorizontal: 15,
          }}>
        {this.state.imageclass}
        </Text>

        </View>
      );
    }
  };
  _maybeRenderUploadingOverlay = () => {
    if (this.state.uploading) {
      return (
        <View
          style={[
            StyleSheet.absoluteFill,
            {
              backgroundColor: 'rgba(0,0,0,0.4)',
              alignItems: 'center',
              justifyContent: 'center',
            },
          ]}>
          <ActivityIndicator color="#fff" animating size="large" />
        </View>
      );
    }
  };

  _maybeRenderImage = () => {
    let { image } = this.state;
    if (!image) {
      return;
    }

    return (
      <View
        style={{
          marginTop: 30,
          width: 250,
          borderRadius: 3,
          elevation: 2,
          shadowColor: 'rgba(0,0,0,1)',
          shadowOpacity: 0.2,
          shadowOffset: { width: 4, height: 4 },
          shadowRadius: 5,
        }}>
        <View
          style={{
            borderTopRightRadius: 3,
            borderTopLeftRadius: 3,
            overflow: 'hidden',
          }}>
            
          
          <Image source={{ uri: image }} style={{ width: 250, height: 250 }} /> 
        </View>

        <Text
          onPress={this._copyToClipboard}
          onLongPress={this._share}
          style={{ paddingVertical: 10, paddingHorizontal: 10 }}>
          {image}
        </Text>
      </View>
    );
  };

  _share = () => {
    Share.share({
      message: this.state.image,
      title: 'Check out this photo',
      url: this.state.image,
    });
  };

  _copyToClipboard = () => {
    Clipboard.setString(this.state.image);
    alert('Copied image URL to clipboard');
  };

  _takePhoto = async () => {
    let pickerResult = await ImagePicker.launchCameraAsync({
      allowsEditing: true,
      aspect: [4, 3],
    });

    this._handleImagePicked(pickerResult);
  };

  _pickImage = async () => {
    let pickerResult = await ImagePicker.launchImageLibraryAsync({
      allowsEditing: true,
      aspect: [4, 3],
    });

    this._handleImagePicked(pickerResult);
  };

  _handleImagePicked = async pickerResult => {
    let uploadResponse, uploadResult;

    try {
      this.setState({ uploading: true });

      if (!pickerResult.cancelled) {
        uploadResponse = await uploadImageAsync(pickerResult.uri);
        uploadResult = await uploadResponse.json();
        this.setState({ image: uploadResult.location });
        if(uploadResult.classification.dog > uploadResult.classification.cat) {
          this.setState({ imageclass: 'Dog' });
        } else {
          this.setState({ imageclass: 'Cat' });
        }
        
      }
    } catch (e) {
      console.log({ uploadResponse });
      console.log({ uploadResult });
      console.log({ e });
      alert('Upload failed, sorry :(');
    } finally {
      this.setState({ uploading: false });
    }
  };
}

async function uploadImageAsync(uri) {
  let apiUrl = 'https://file-upload-example-backend-dkhqoilqqn.now.sh/upload';

  // Send image to our API
  
  // Change text based on API response
  
  
  
  // Note:
  // Uncomment this if you want to experiment with local server
  //
  // if (Constants.isDevice) {
  //   apiUrl = `https://your-ngrok-subdomain.ngrok.io/upload`;
  // } else {
  //   apiUrl = `http://localhost:3000/upload`
  // }


  //let uriParts = uri.split('.');
  let fileType = uri[uri.length - 1];

  let formData = new FormData();
  formData.append('photo', {
    uri,
    name: `photo.${fileType}`,
    type: `image/${fileType}`,
  });

  let options = {
    method: 'POST',
    body: formData,
    headers: {
      Accept: 'application/json',
      'Content-Type': 'multipart/form-data',
    },
  };

  return fetch(apiUrl, options);
}
